<a href="https://colab.research.google.com/github/ewertonhseixas/curso_github/blob/main/alura%2Bgoogle_gemini_projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip -q install google-genai

In [5]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [7]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Mestre Narrativo --- #
##########################################
def agente_narrador(tema, n_jogadores, dificuldade):

    narrador = Agent(
        name="agente_narrador",
        model="gemini-2.0-flash",
        instruction="""
        Você é o Mestre Narrativo focado em Dungeons and Dragons 5E, é o coração criativo da one-shot, responsável
        por tecer a trama da aventura a partir das informações iniciais fornecidas
        (número de jogadores, tema, nível de dificuldade). Você conceberá
        uma história épica e com leve tom de humor quando necessário, definirá os locais a serem explorados, criará variados NPCs com
        suas motivações e diálogos (NÃO USAR O NOME ELARA), e estruturará a progressão da aventura
        em até 4 encontros distintos, culminando em um final que deixe portas
        abertas para o futuro.
        Suas responsabilidades são:
        Receber as informações iniciais: número de jogadores,
        tema central da aventura e nível de dificuldade desejado.
        Criar a história da one-shot, incluindo:
        Uma introdução envolvente para situar os jogadores.
        Até 4 encontros distintos, que podem ser combates, desafios de
        perícia, interações sociais ou enigmas.
        Descrição detalhadas dos locais importantes, NPCs chave (com suas personalidades
        e motivações), e objetos relevantes para a trama.
        Um final aberto que deixe espaço para possíveis continuações.
        Definir os objetivos principais dos jogadores dentro da aventura.
        """,
        description="Agente narrativo que cria a história e seus desafios",
        tools=[google_search]
    )

    entrada_do_agente_narrador = f"Tema: {tema}\nNúmero de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}"

    historia = call_agent(narrador, entrada_do_agente_narrador)
    return historia

In [12]:
################################################
# --- Agente 2: Mestre de Encontros (Balanceamento) --- #
################################################
def agente_balanceador(n_jogadores, dificuldade, historia_inicial):
    balanceador = Agent(
        name="agente_balanceador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é o Mestre de Encontros atuará como o balanceador de desafios da
        aventura, pegando a visão narrativa traduzindo-a em encontros práticos
        e adequados ao nível do grupo de jogadores. Para combates, selecionará e
        ajustará criaturas do bestiário, além de calcular o nível de ameaça do encontro.
        Para outros desafios, definirá as perícias necessárias, as CDs e as possíveis consequências,
        garantindo que cada obstáculo seja desafiador, mas não intransponível.
        Responsabilidades:
        Receber a história inicial e a informação sobre o número de jogadores
        e o Nível de Dificuldade.
        Analisar cada encontro proposto.
        Para encontros de combate:
        Selecionar criaturas apropriadas do bestiário de D&D 5e.
        Ajustar o número de criaturas para garantir um desafio adequado ao grupo (considerando o nível de dificuldade desejado).
        Calcular o Nível de Encontro (Encounter Level - EL) ou o Valor de Pontos
        de Experiência (XP Thresholds) para cada combate, garantindo que esteja
        dentro da faixa desejada de dificuldade (Fácil, Médio, Difícil, Mortal).
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Descrever armadilhas e obstáculos do terreno.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        Sugerir possíveis consequências para sucesso e falha.
        No caso de enigmas, fornecer a descrição do enigma e as possíveis soluções.
        """,
        description="Agente que balancea todos os desafios da aventura para a quantidade de jogadores e Nível de Dificuldade",
        tools=[google_search]
    )

    entrada_do_agente_balanceador = f"Número de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}\nHistória Inicial: {historia_inicial}"
    # Executa o agente
    balanceamento = call_agent(balanceador, entrada_do_agente_balanceador)
    return balanceamento

In [13]:
######################################
# --- Agente 3: Mestre de Contingências (Soluções Alternativas) --- #
######################################
def agente_solucao(tema, historia_inicial, balanceamento):
    solucao = Agent(
        name="agente_solucao",
        model="gemini-2.0-flash",
        instruction="""
            Você será o Mestre de Contingências terá um olhar estratégico sobre
            a aventura, antecipando os imprevistos que podem surgir durante o jogo.
            Você oferecerá soluções alternativas para desvios na narrativa, como
            escolhas inesperadas dos jogadores ou falhas críticas, e fornecerá
            dicas rápidas para lidar com situações delicadas, como a derrota de
            um personagem, garantindo que a sessão continue fluida e envolvente
            apesar dos percalços.
            Responsabilidades:
            Receber a história e as informações dos encontros balanceados.
            Antecipar possíveis desvios da narrativa planejada (e.g., jogadores
            ignoram uma pista importante, tomam uma decisão inesperada).
            Sugerir soluções alternativas para manter a aventura interessante e
            progredindo, mesmo que os jogadores não sigam o caminho "ideal".
            Oferecer dicas rápidas para lidar com situações como a morte de um
            personagem (e.g., introduzir um novo NPC aliado, oferecer uma forma
             de ressurreição com um custo).
            Pensar em ramificações da história caso os jogadores façam escolhas significativas.
            """,
        description="Agente de contingência para auxiliar o mestre oferencendo alternativas",
        tools=[google_search]
    )
    entrada_do_agente_solucao = f"Tema: {tema}\nHistória Inicial: {historia_inicial}\nBalanceamento: {balanceamento}"
    # Executa o agente
    alternativas = call_agent(solucao, entrada_do_agente_solucao)
    return alternativas

In [14]:
##########################################
# --- Agente 4: Mestre Organizador (Revisão e Formatação) --- #
##########################################
def agente_revisor(historia_inicial, balanceamento, alternativas):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é o Mestre Organizador e será o curador final da informação,
            recebendo o conteúdo bruto dos demais agentes e aprimorando-o e detalhando
            para facilitar a sua condução da sessão. Você revisará a clareza e a
            consistência de todo o material, organizando-o de forma lógica e
            sequencial, destacando informações cruciais como falas de NPCs e
            detalhes dos encontros, para que o mestre tenha tudo o que precisa
            ao seu alcance de maneira otimizada.
            Responsabilidades:
            Receber todas as informações geradas até o momento (historia
            inicial, balanceamento, alternativas)
            Revisar o conteúdo para garantir clareza, consistência e fluidez.
            Organizar as informações de forma lógica, sequencial e detalhada
            para facilitar a consulta durante a sessão de jogo.
            Uma possível estrutura seria:
            Cabeçalho: Título da Aventura, número de jogadores, ND da Aventura,
            Introdução: Cenário inicial, gancho da aventura.
            Encontro 1: Descrição detalhada do local (terreno, armadilhas, enigmas, combate, testes sociais), NPCs envolvidos (com falas
            importantes destacadas)
            Encontro 2: (Mesma estrutura do Encontro 1)
            Encontro 3 (opcional): (Mesma estrutura)
            Encontro 4 (opcional): (Mesma estrutura)
            Possível Final: Descrição do desfecho imediato e ganchos para continuações.
            Soluções Alternativas e Dicas: Lista de sugestões para lidar com imprevistos.
            Formatar o texto de maneira clara e concisa e sempre descrevendo detalhadamente, utilizando títulos,
            subtítulos e formatação para destacar informações importantes (e.g., falas de NPCs em itálico).
            """,
        description="Agente revisor e formatador da aventura",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"História Inicial: {historia_inicial}\nBalanceamento: {balanceamento}\nAlternativas: {alternativas}"
    # Executa o agente
    historia_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return historia_revisada

In [15]:
################################################
# --- Agente 5: Mestre de Regras --- #
################################################
def agente_regras(n_jogadores, dificuldade, historia_revisada):
    regras = Agent(
        name="agente_regras",
        model="gemini-2.0-flash",
        instruction="""
        Você é o Mestre de Regras atuará como o buscador das regras oficiais e
        aplicará em toda a aventura. Tratará as estatísticas de monstros e criaturas do bestiário,
        armadilhas, testes de habilidades, perícias e itens. Faça buscas em sites e foruns confiáveis
        para Dungeons And Dragons 5E.
        Responsabilidades:
        Receber a história revisada e a informação sobre o número de jogadores
        e o Nível de Dificuldade.
        Analisar cada encontro proposto.
        Calcular CD de todos os testes de perícia corretamente.
        Para encontros de combate:
        Detalhar as estatisticas das criaturas do bestiário de D&D 5e dessa forma:
        Nome - tipo - tamanho - alinhamento - ND
        CA - HP - Movimento - Resistências/Fraquezas
        Ataques - Habilidades - Ações Especiais
        Ajustar o número de criaturas ou suas estatísticas (se necessário) para
        garantir um desafio adequado ao grupo (considerando o nível de dificuldade desejado).
        Calcular o Nível de Encontro (Encounter Level - EL) ou o Valor de Pontos
        de Experiência (XP Thresholds) para cada combate, garantindo que esteja
        dentro da faixa desejada de dificuldade (Fácil, Médio, Difícil, Mortal).
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Detalhar as armadilhas, a CD necessária para evitá-las e as possíveis consequências.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        Sugerir possíveis consequências para sucesso e falha.
        No caso de enigmas, fornecer a descrição do enigma e as possíveis soluções.
        """,
        description="Agente que confirma todas as regras para a história",
        tools=[google_search]
    )

    entrada_do_agente_regras = f"Número de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}\nHistória Revisada: {historia_revisada}"
    # Executa o agente
    regras = call_agent(regras, entrada_do_agente_regras)
    return regras


In [16]:
################################################
# --- Agente 6: Edição Final --- #
################################################
def agente_final(regras, historia_revisada):
    final = Agent(
        name="agente_final",
        model="gemini-2.0-flash",
        instruction="""
        Você é o editor e revisor final, receberá a história e todas as regras aplicadas a ela.
        Garanta que ela seja rica em detalhes, organizada na ordem dos seus desafios e bem estruturada.
        Não economize no detalhamento da história, aumente se necessário, mas sem alterar as regras.
        Manter a estrutura inicial:
        Uma possível estrutura seria:
        Cabeçalho: Título da Aventura, número de jogadores, ND da Aventura,
        Introdução: Cenário inicial, gancho da aventura.
        Encontro 1: Descrição detalhada do local (terreno, armadilhas, enigmas, combate, testes sociais), quantidade de inimigos e  NPCs envolvidos (com falas
        importantes destacadas)
        Encontro 2: (Mesma estrutura do Encontro 1)
        Encontro 3 (opcional): (Mesma estrutura)
        Encontro 4 (opcional): (Mesma estrutura)
        Detalhar as fichas de estatisticas das criaturas do bestiário de D&D 5e dessa forma e a quantidade de inimigos em cada encontro sempre que aparecer:
        Nome - tipo - tamanho - alinhamento - ND - CA - HP - Movimento - Resistências/Fraquezas - Ataques - Habilidades - Ações Especiais
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Detalhar as armadilhas, a CD necessária para evitá-las e as possíveis consequências.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        """,
        description="Agente de edição final",
        tools=[google_search]
    )

    entrada_do_agente_final = f"Regras: {regras}\nHistória Revisada: {historia_revisada}"
    # Executa o agente
    final = call_agent(final, entrada_do_agente_final)
    return final


In [17]:
print("Assistente do Mestre de D&D 🧙‍♂️")

# --- Obter os dados do Usuário ---
tema = input("❓ Por favor, digite o TEMA CENTRAL da sua aventura \nExemplo: salvar o reino de um mal iminente, resgatar um tesouso perdido, proteger uma caravana importante \nQual será o seu tema? ")
n_jogadores = input("❓ Agora me diga quantos jogadores participarão da aventura? ")
dificuldade = input("❓ Qual o Nível de Dificuldade desejada para sua aventura? \n(Exemplo: ND4 para 4 jogadores de nível 1) - ")
# Inserir lógica do sistema de agentes ################################################
if not tema:
    print("Você esqueceu de digitar o tema!")
else:
    print(f"O tema da sua aventura será: {tema}")

if not n_jogadores:
    print("Você esqueceu de digitar o número de jogadores!")
else:
    print(f"O número de jogadores será: {n_jogadores}")

if not dificuldade:
    print("Você esqueceu de digitar o nível de dificuldade!")
else:
    print(f"O nível de dificuldade será: {dificuldade}")

    historia_inicial = agente_narrador(tema, n_jogadores, dificuldade)
    print("\n--- Esta sendo gerada sua grande aventura! ---\n")
    print("--------------------------------------------------------------")

    balanceamento = agente_balanceador(n_jogadores, dificuldade, historia_inicial)
    print("\n--- Balanceando a Aventura ---\n")
    print("--------------------------------------------------------------")

    alternativas = agente_solucao(tema, historia_inicial, balanceamento)
    print("\n--- Criando Planos de Contigência (Alternativas) ---\n")
    print("--------------------------------------------------------------")

    historia_revisada = agente_revisor(historia_inicial, balanceamento, alternativas)
    print("\n--- Revisando a História ---\n")
    print("--------------------------------------------------------------")

    regras = agente_regras(n_jogadores, dificuldade, historia_revisada)
    print("\n--- Aplicando Regras da Aventura ---\n")
    print("--------------------------------------------------------------")

    final = agente_final(regras, historia_revisada)
    print("\n--- Aventura Final ---\n")
    display(to_markdown(final))
    print("--------------------------------------------------------------")


Assistente do Mestre de D&D 🧙‍♂️
❓ Por favor, digite o TEMA CENTRAL da sua aventura 
Exemplo: salvar o reino de um mal iminente, resgatar um tesouso perdido, proteger uma caravana importante 
Qual será o seu tema? horror na vila
❓ Agora me diga quantos jogadores participarão da aventura? 4
❓ Qual o Nível de Dificuldade desejada para sua aventura? 
(Exemplo: ND4 para 4 jogadores de nível 1) - ND4
O tema da sua aventura será: horror na vila
O número de jogadores será: 4
O nível de dificuldade será: ND4

--- Esta será sua grande aventura! ---

--------------------------------------------------------------

--- Balanceamento da Aventura ---

--------------------------------------------------------------

--- Plano de Contigência (Alternativas) ---

--------------------------------------------------------------

--- História Revisada ---

--------------------------------------------------------------

--- Regras da Aventura ---

--------------------------------------------------------------

> Aqui está a aventura "A Praga da Colheita Sombria" revisada e expandida, pronta para ser aterrorizante e divertida!
> 
> **Título:** A Praga da Colheita Sombria
> 
> **Número de Jogadores:** 4
> 
> **Nível de Dificuldade (ND):** 4 (Médio a Difícil)
> 
> **Introdução:**
> 
> A aventura começa com os personagens a cavalo ou a pé, aproximando-se da isolada vila de Vale Verde [i]. O ar fica mais pesado à medida que se aproximam, um odor sutilmente pútrido pairando no ar, prenunciando a desgraça que se abateu sobre o lugar [i]. Vale Verde, normalmente um oásis de prosperidade agrícola, agora se encontra definhando sob o peso de uma praga inexplicável [i]. As colheitas, outrora abundantes, estão agora apodrecendo nos campos, retorcidas e enegrecidas por uma doença grotesca [i]. Os animais, antes saudáveis e robustos, jazem mortos, seus corpos inchados e cobertos de manchas escuras [i].
> 
> Os moradores, outrora conhecidos por sua hospitalidade e alegria, agora se escondem em suas casas, seus rostos marcados pelo medo e pela desconfiança [i]. Poucos se aventuram nas ruas desertas, e aqueles que o fazem lançam olhares sombrios e sussurram sobre "a maldição" e "o fim dos tempos" [i]. A atmosfera opressiva é palpável, um silêncio sinistro interrompido apenas pelo gemido ocasional do vento uivante [i].
> 
> O grupo foi convocado a Vale Verde por Frei Tuck, um clérigo benevolente e respeitado, que serviu a vila por muitos anos [i]. Em uma carta urgente, ele expressa sua profunda preocupação com a praga, temendo que ela seja apenas um sintoma de um mal muito mais profundo e antigo que se esconde sob a vila [i]. Ele implora aos aventureiros que venham em seu auxílio, buscando sua coragem e habilidade para desvendar a verdade por trás da praga e restaurar a paz em Vale Verde [i].
> 
> **Objetivos dos Jogadores:**
> 
> *   Investigar a causa da praga que assola Vale Verde, explorando os campos amaldiçoados, interrogando os moradores assustados e buscando pistas em locais sombrios e esquecidos [i].
> *   Encontrar uma maneira de deter a praga e salvar os moradores da fome e da desespero, buscando uma cura milagrosa, destruindo a fonte da doença ou encontrando uma solução criativa e inesperada [i].
> *   Investigar os rumores de um mal ancestral adormecido sob a vila, desenterrando segredos há muito enterrados, enfrentando horrores indescritíveis e impedindo seu despertar para evitar que a escuridão consuma Vale Verde e se espalhe pelo mundo [i].
> 
> **Encontro 1: A Chegada Tensa**
> 
> *   **Descrição:** Ao chegarem a Vale Verde, os personagens são recebidos não com a calorosa hospitalidade que esperavam, mas com olhares desconfiados e hostilidade velada [i]. Os aldeões, liderados por um homem corpulento e carrancudo chamado Barnaby, os confrontam na entrada da vila, bloqueando seu caminho com carrinhos de mão e ferramentas agrícolas [i]. Seus rostos estão marcados pela exaustão e pelo medo, e seus olhos brilham com uma mistura de desespero e ressentimento [i].
> 
> *   **NPCs Envolvidos:**
>     *   **Barnaby:** O líder da vila, um homem rude e desconfiado, mas profundamente preocupado com o bem-estar de seu povo [i]. Ele vê os forasteiros como uma ameaça potencial, temendo que eles tragam ainda mais infortúnio para Vale Verde [i].
>         *   *Fala Sugerida:* "Quem são vocês e o que querem aqui? Já temos problemas demais sem vocês se meterem!"
>     *   **Frei Tuck:** O clérigo bondoso e compassivo, que secretamente encontra os personagens nos arredores da vila, longe dos olhares curiosos dos aldeões [i]. Ele está aliviado com a chegada dos aventureiros, mas também preocupado com a crescente histeria e desconfiança que se espalham entre os moradores [i].
>         *   *Fala Sugerida:* "Graças aos céus, vocês vieram! A situação é ainda mais terrível do que eu temia. A praga está consumindo nossas colheitas e nossa esperança."
> 
> *   **Interação:**
>     *   Um teste de Carisma (Intimidação ou Persuasão) ND 13 é necessário para convencer os aldeões de que os aventureiros estão ali para ajudar e não para causar mais problemas [i].
>         *   **Sucesso:** Os aldeões ficam mais receptivos e dispostos a compartilhar informações sobre a praga, seus efeitos devastadores e os rumores perturbadores que circulam pela vila [i]. Eles podem revelar detalhes sobre a fazenda da família Harvester, o local onde a praga parece ter se originado, e os estranhos rituais que Silas Harvester, o chefe da família, tem realizado ultimamente [i].
>         *   **Falha:** Os personagens terão que recorrer a outros meios para obter informações, como usar a força para intimidar os aldeões, suborná-los com promessas de riqueza ou usar a astúcia para espioná-los e descobrir seus segredos [i]. Eles também podem tentar encontrar Frei Tuck por conta própria, seguindo pistas sutis ou buscando ajuda de algum aldeão mais amigável e disposto a cooperar [i].
> 
> *   **Alternativas:**
>     *   Se os jogadores não conseguirem convencer os aldeões ou encontrarem Frei Tuck, o clérigo pode entrar em contato com eles secretamente, enviando uma mensagem por meio de um corvo ou de um mensageiro confiável [i].
>     *   Se os jogadores ignorarem os aldeões e partirem para investigar a vila por conta própria, eles podem se deparar com encontros menores que forneçam pistas sobre a praga, como encontrar animais mortos e mutilados, testemunhar rituais secretos ou ouvir sussurros perturbadores vindo das casas fechadas [i].
> 
> **Encontro 2: A Investigação na Fazenda Corrompida**
> 
> *   **Descrição:** A fazenda da família Harvester, outrora um símbolo de prosperidade e abundância, agora se encontra em um estado de decadência e corrupção [i]. Os campos estão cobertos de plantas retorcidas e enegrecidas, seus frutos apodrecendo nas videiras [i]. O ar está denso com um cheiro nauseante de enxofre e carne podre, tornando difícil respirar [i]. Animais mortos jazem espalhados por toda parte, seus corpos inchados e cobertos de manchas escuras e pústulas [i].
> 
> No centro da fazenda, ergue-se um celeiro dilapidado, suas paredes rachadas e seus telhados desabando [i]. Do interior, emanam cânticos estranhos e sussurros macabros, acompanhados pelo brilho sinistro de velas tremeluzentes [i]. Ao se aproximarem cautelosamente, os personagens podem vislumbrar, através das frestas nas paredes, um grupo de cultistas encapuzados reunidos em torno de um altar improvisado, realizando um ritual profano para "alimentar a terra" com oferendas sangrentas [i]. Liderando o ritual, está Silas Harvester, o chefe da família, um homem magro e de olhos fanáticos, vestido com um manto esfarrapado e adornado com símbolos macabros [i].
> 
> *   **NPCs Envolvidos:**
>     *   **Silas Harvester:** O líder dos cultistas, um fanático obcecado em despertar um mal ancestral adormecido sob a vila [i]. Ele acredita que a praga é um sinal da chegada da "Era da Escuridão" e que, ao realizar rituais profanos, ele pode acelerar o processo e trazer o caos e a destruição ao mundo [i].
>         *   *Fala Sugerida:* "A escuridão está se erguendo! A terra está sedenta por sangue! Em breve, todos vocês se curvarão diante do poder do Senhor das Trevas!"
> 
> *   **Combate:**
>     *   Cultistas (3): Utilizar as estatísticas de **Acólito** do Livro do Mestre (MM p. 12) [i].
>         *   **Acólito** - Humanoide (Humano) - Médio - Neutro Maligno - ND 1/4
>             *   CA 10 - 9 HP - Movimento 30ft
>             *   Ataques: Maça (corpo a corpo) +2 para acertar, 1d6+2 dano de concussão
>             *   Habilidades: Magia Divina (CD 10)
>     *   Silas Harvester: Utilizar as estatísticas de **Fanático** do Livro do Mestre (MM p. 150), com a habilidade adicional de lançar a magia *Praga* uma vez por dia (CD 13 para resistência de Constituição) [i].
>         *   **Fanático** - Humanoide (Humano) - Médio - Neutro Maligno - ND 2
>             *   CA 13 (cota de malha) - 33 HP - Movimento 30ft
>             *   Resistências: Dano necrótico
>             *   Ataques: Cimitarra (corpo a corpo) +4 para acertar, 1d6+2 dano cortante; Besta leve (à distância) +4 para acertar, 1d8+2 dano perfurante
>             *   Habilidades: Magia Divina (CD 11)
>             *   Ações Especiais: Frenesi Fanático (vantagem em testes de ataque corpo a corpo)
> 
> *   **Ambiente:**
>     *   Plantações retorcidas, animais mortos e cheiro de enxofre [i].
>     *   Celeiro com passagens secretas (CD 14 para notar) escondidas atrás de pilhas de feno ou debaixo do piso de madeira [i]. As passagens levam a um sistema de túneis subterrâneos que se estendem sob a vila [i].
> 
> *   **Táticas:**
>     *   Os cultistas tentam flanquear os personagens e usar ataques coordenados, buscando protegendo Silas Harvester e permitindo que ele continue o ritual [i].
>     *   Silas Harvester prioriza o uso da magia *Praga* nos personagens mais fortes fisicamente, buscando enfraquecê-los e torná-los mais vulneráveis aos ataques dos cultistas [i].
>     *   Os cultistas usam o ambiente para criar cobertura e dificultar o movimento dos personagens, escondendo-se atrás de pilhas de feno e usando as paredes do celeiro como proteção [i].
> 
> *   **Alternativas:**
>     *   Se Silas Harvester for derrotado rapidamente, ele pode tentar fugir para os túneis subterrâneos, levando os personagens a persegui-lo e a explorar o sistema de cavernas sob a vila [i].
>     *   Se os jogadores forem derrotados, eles são capturados pelos cultistas e levados para os túneis subterrâneos, onde são amarrados e deixados para serem sacrificados ao Senhor das Trevas [i]. Eles podem escapar mais tarde, talvez com a ajuda de um aldeão arrependido ou com a intervenção de Frei Tuck [i].
> 
> **Encontro 3: Nas Profundezas da Terra**
> 
> *   **Descrição:** As passagens secretas no celeiro levam a um labirinto de túneis subterrâneos úmidos e escuros, que se estendem sob a vila [i]. As paredes estão cobertas de musgo e fungos bioluminescentes, que emitem uma luz fraca e sinistra [i]. O ar está frio e úmido, e um cheiro pungente de mofo e umidade impregna tudo [i]. Teias de aranha densas cobrem os túneis, e o chão está repleto de ossos de animais e outros detritos [i].
> 
> À medida que os personagens se aprofundam nos túneis, eles começam a ouvir o som de passos rastejantes e o farfalhar de inúmeras pernas [i]. Logo, eles se deparam com uma horda de aranhas gigantes e enxames de aranhas, que os atacam em busca de comida [i].
> 
> No final dos túneis, os personagens encontram uma câmara cavernosa, iluminada por uma luz fraca e pulsante vinda de um poço de escuridão no centro da sala [i]. Diante do poço, ergue-se um altar de pedra com símbolos sinistros gravados em sua superfície [i]. Uma energia sinistra emana do altar, fazendo com que os pelos da nuca dos personagens se arrepiem [i].
> 
> *   **Combate:**
>     *   Aranhas Gigantes (2): Estatísticas do Livro dos Monstros (MM p. 328) [i].
>         *   **Aranha Gigante** - Besta - Grande - Neutro - ND 1
>             *   CA 14 (armadura natural) - 26 HP - Movimento 30ft, escalar 30ft
>             *   Sentidos: Visão no escuro 60ft, Sentido de tremores 30ft
>             *   Ataques: Mordida (corpo a corpo) +5 para acertar, 1d8+3 dano perfurante + 1d4 veneno (CD 11 Constituição)
>             *   Habilidades: Escalada
>     *   Enxame de Aranhas: Estatísticas do Livro dos Monstros (MM p. 337) [i].
>         *   **Enxame de Aranhas** - Besta - Médio - Neutro - ND 1/2
>             *   CA 12 - 22 HP - Movimento 20ft, escalar 20ft
>             *   Resistências: Dano de concussão, perfurante e cortante contra ataques não mágicos
>             *   Vulnerabilidades: Dano de área
>             *   Ataques: Mordida (corpo a corpo) +3 para acertar, 2d6 dano perfurante + 1d4 veneno (CD 10 Constituição)
>             *   Habilidades: Escalada
> 
> *   **Desafio Ambiental:**
>     *   Um teste de resistência de Sabedoria (CD 13) é necessário ao tocar no altar [i].
>         *   **Falha:** O personagem é atormentado por alucinações aterrorizantes e visões perturbadoras, sofrendo desvantagem em todas as jogadas de ataque, testes de habilidade e testes de resistência por 1d4 turnos [i].
> 
> *   **Ambiente:**
>     *   Túneis subterrâneos úmidos e escuros, cobertos de teias de aranha e repletos de ossos e detritos [i].
>     *   Câmara cavernosa com um altar de pedra e um poço de escuridão pulsante [i].
> 
> *   **Táticas:**
>     *   As aranhas gigantes tentam agarrar os personagens com suas teias pegajosas e arrastá-los para as profundezas dos túneis, onde podem devorá-los à vontade [i].
>     *   O enxame de aranhas ataca os personagens mais vulneráveis, cercando-os e mordendo-os com suas presas venenosas [i].
>     *   As aranhas usam o ambiente para dificultar o movimento dos personagens, escondendo-se nas sombras e emboscando-os em cantos escuros [i].
> 
> *   **Alternativas:**
>     *   Se os jogadores ficarem presos nas teias das aranhas gigantes, eles podem tentar se libertar usando testes de Força (Atletismo) ou Destreza (Acrobacia) [i].
>     *   Se os jogadores se recusarem a tocar no altar, ele pode emanar uma onda de energia sombria que os força a fazer o teste de resistência de Sabedoria [i].
> 
> **Encontro 4: O Coração da Escuridão**
> 
> *   **Descrição:** No centro do poço de escuridão, os personagens confrontam a fonte da praga: um fragmento corrompido de um antigo deus esquecido, aprisionado ali há eras [i]. O fragmento se manifesta como um **Horror Tentacular**, uma abominação grotesca feita de carne retorcida e tentáculos pulsantes [i]. Sua presença emana uma aura de terror e desespero, fazendo com que os corações dos personagens batam mais forte e suas mentes se encham de visões perturbadoras [i].
> 
> *   **Inimigo:**
>     *   Horror Tentacular: Use as estatísticas de uma **Aberração** de ND 4, ajustando-as para refletir sua natureza corrompida e tentacular [i]. Exemplo:
>         *   **Aberração Tentacular** - Aberração - Grande - Caótico Maligno - ND 4
>             *   CA 14 (armadura natural) - 75 HP - Movimento 30ft
>             *   Ataques: Tentáculos (corpo a corpo) +6 para acertar, 2d6+4 dano de concussão + 1d6 dano necrótico (CD 13 Constituição para evitar ficar paralisado por 1 turno).
>             *   Habilidades: Resistência à magia
> 
> *   **Desafio de Resistência:**
>     *   Testes de Sabedoria (CD 15) são necessários para resistir aos sussurros e visões perturbadoras que emanam do fragmento [i].
>         *   **Falha:** O personagem é dominado pelo medo, pela confusão ou pela loucura, sofrendo efeitos debilitantes que o tornam mais vulnerável aos ataques do Horror Tentacular [i]. Os efeitos podem incluir:
>             *   **Medo:** O personagem fica amedrontado e deve gastar seu turno tentando fugir do Horror Tentacular [i].
>             *   **Confusão:** O personagem fica confuso e ataca aleatoriamente a criatura mais próxima, aliada ou inimiga [i].
>             *   **Loucura:** O personagem sofre uma crise de insanidade temporária, agindo de forma irracional e imprevisível [i].
> 
> *   **Ambiente:**
>     *   O centro do poço de escuridão está impregnado de energia negativa, fazendo com que a temperatura caia drasticamente e o ar se torne pesado e difícil de respirar [i].
>     *   O chão está coberto de uma gosma escura e viscosa, que dificulta o movimento e emite um cheiro nauseante [i].
>     *   As paredes da câmara estão cobertas de símbolos sinistros e gravuras grotescas, que parecem pulsar com uma energia maligna [i].
> 
> *   **Táticas:**
>     *   O Horror Tentacular usa seus tentáculos para atacar os personagens à distância, agarrando-os e esmagando-os com sua força bruta [i].
>     *   Ele usa os sussurros e visões para distrair e enfraquecer os personagens, aproveitando-se de seus medos e inseguranças [i].
>     *   O Horror Tentacular tenta corromper a mente dos personagens, oferecendo-lhes poder e conhecimento em troca de sua lealdade [i].
> 
> *   **Condições de Vitória:**
>     *   Para derrotar o Horror Tentacular e purificar o poço de escuridão, os personagens devem destruir o altar de pedra e lançar água benta no poço [i]. Frei Tuck pode ter fornecido a água benta antes de sua partida, ou os personagens podem ter que encontrar uma fonte alternativa nas cavernas, como uma nascente sagrada ou um relicário escondido [i].
> 
> *   **Alternativas:**
>     *   Se os jogadores não conseguirem obter água benta, eles podem ter a chance de encontrar uma arma mágica ou um artefato sagrado que possa ser usado para destruir o Horror Tentacular [i].
>     *   Se os jogadores decidirem não destruir o fragmento corrompido, ele começará a se fortalecer, causando efeitos cada vez mais graves na vila e liberando hordas de monstros para atacar os moradores [i].
> 
> **Possível Final:**
> 
> Se os personagens conseguirem destruir o fragmento corrompido e purificar o poço de escuridão, a praga desaparecerá gradualmente e Vale Verde começará a se recuperar [i]. As colheitas voltarão a florescer, os animais se recuperarão e os moradores recuperarão sua alegria e esperança [i].
> 
> Os aldeões serão eternamente gratos aos heróis que os salvaram da escuridão [i]. Barnaby, o líder da vila, pedirá desculpas por sua desconfiança inicial e prometerá reconstruir a vila em homenagem aos aventureiros [i].
> 
> Frei Tuck revelará que o fragmento corrompido era apenas uma parte de um ser muito maior e mais poderoso, um antigo deus esquecido que foi aprisionado sob a terra há eras [i]. Ele explicará que outros fragmentos podem estar espalhados pelo mundo, esperando para serem encontrados e corrompidos pela escuridão [i].
> 
> A aventura termina com a possibilidade de uma nova campanha, na qual os personagens embarcam em uma jornada épica para encontrar os outros fragmentos corrompidos e impedir que o antigo deus esquecido retorne e mergulhe o mundo na escuridão eterna [i].
> 
> **Soluções Alternativas e Dicas:**
> 
> *   **Morte de um Personagem:** Se um personagem morrer durante a aventura, introduza um novo NPC aliado que possa se juntar ao grupo e ajudá-los a completar sua missão [i]. Alternativamente, ofereça aos jogadores uma forma de ressurreição com um custo, como fazer um pacto com uma entidade sombria ou realizar um ritual perigoso [i].
> 
> *   **Pista Ignorada:** Se os jogadores ignorarem uma pista importante, faça com que ela reapareça em outro local ou seja mencionada por um NPC diferente [i]. Você também pode adaptar os encontros futuros para incorporar a pista ignorada, forçando os jogadores a enfrentarem as consequências de sua falta de atenção [i].
> 
> *   **Decisão Inesperada:** Se os jogadores tomarem uma decisão inesperada que mude o curso da aventura, use essa decisão como uma oportunidade para explorar novos aspectos da vila e da praga [i]. Adapte os encontros futuros para refletir a nova direção da história, permitindo que os jogadores moldem o mundo ao seu redor com suas escolhas [i].
> 
> Lembre-se de adaptar os detalhes e desafios da aventura de acordo com o seu grupo, personalizando a experiência para atender aos seus gostos e preferências [i]. Use a música e a ambientação para criar uma atmosfera imersiva, transportando os jogadores para o mundo sombrio e misterioso de Vale Verde [i]. E, o mais importante, deixe os jogadores tomarem decisões e moldarem a história, permitindo que eles se sintam verdadeiramente no controle de seu destino [i]. O mais importante é que você e seus jogadores se divirtam!
> 


--------------------------------------------------------------
